In [ ]:
import sys
import os
import numpy as np
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path + "/src/simulations_v2")
    sys.path.append(module_path + "/src/simulations_v2/params/spring_2021_calibration/")
    sys.path.append(module_path + "/notebooks/pnas_paper_figs/")

from multiprocessing import Process


In [ ]:
!pip install pyDOE

In [ ]:
from uncertainty_analysis import calculate_pessimistic_scenario, \
            load_sim_output, residential_regression, virtual_vs_residential_regression,\
            UNCERTAINTY_PARAMS_LIST

from LHS_sim_code import *

In [ ]:
quantiles = np.arange(0.05,1,0.05)
quantiles

In [ ]:
uncertainty_params_list = UNCERTAINTY_PARAMS_LIST

In [ ]:
# 100 trajectories for each of 1000 configs sampled from the prior is stored under 
# notebooks/sept_29_prior_sims/

In [ ]:
lhs_output_sim_files = []
for i in range(2000):
    #fname = '/home/jmc678/covid_data/group-testing/notebooks/apr_29_scenarios/point_{}.dill'.format(i)
    
    fname = '/Users/brianliu/GraduateSchool/Fall2021/MiscResearch/COVID/group-testing/notebooks/apr_29_scenarios/point_{}.dill'.format(i)
    lhs_output_sim_files.append(fname)

scenario_data = load_sim_output(lhs_output_sim_files)
res_results = residential_regression(scenario_data)
#res_results = virtual_vs_residential_regression(scenario_data)
res_pessimistic = calculate_pessimistic_scenario(res_results)
    

In [ ]:
calculate_pessimistic_scenario(res_results, q = 0.95)

In [ ]:
nsamples = 1000

for q in quantiles:

    processes = []
    
    pess_config = calculate_pessimistic_scenario(res_results, q = q)
    param_input = [pess_config[key] for key in uncertainty_params_list]
    
    for i in range(nsamples):
        p = run_new_process_residential_only(param_input, 'sept_30_linear_pess_sims/'+str(q)+'_quantile/trajectory_'+str(i)+'.dill', i)
        processes.append(p)

    print("launched {} processes".format(len(processes)))
    for p in processes:
        p.join()

### Processing Prior Sim

In [ ]:
pd.set_option('display.max_columns', None)  

In [ ]:
lhs_output_sim_files = []
for i in range(999):
    #fname = '/home/jmc678/covid_data/group-testing/notebooks/apr_29_scenarios/point_{}.dill'.format(i)
    if i != 524:
    
        fname = '/Users/brianliu/GraduateSchool/Fall2021/MiscResearch/COVID/group-testing/notebooks/sept_29_prior_sims/point_{}.dill'.format(i)
        lhs_output_sim_files.append(fname)


scenario_data = load_sim_output(lhs_output_sim_files)
res_results = residential_regression(scenario_data)


In [ ]:

nsamples = 10
quantiles = np.arange(0.05,1,0.05)

for q in quantiles:
    print(q)
    pess_config = calculate_pessimistic_scenario(res_results, q = q)
    param_input = [pess_config[key] for key in uncertainty_params_list]
    process=[]
    for i in range(nsamples):
        p = run_new_process_residential_only(param_input, 'sept_29_prior_sims/'+str(q)+'_quantile/trajectory_'+str(i)+'.dill', i)
        processes.append(p)
    

In [ ]:
for p in processes:
        p.join()